<a href="https://colab.research.google.com/github/DieLeitter/ExData_Plotting1/blob/master/matrix3_day5_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install hyperopt

In [33]:
import pandas as pd
import numpy as np

import os 
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical


%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [17]:
cd '/content/drive/My Drive/Colab Notebooks/matrix_three/dw_matrix_road_sign/'

/content/drive/My Drive/Colab Notebooks/matrix_three/dw_matrix_road_sign


In [18]:
ls

data/  LICENSE  logs/  README.md


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')


X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:] # input 
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
    model.compile(loss = 'categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    logdir = os.path.join("logs", datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    tensorboard_callback =tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(
        X_train, 
        y_train,
        batch_size=params_fit.get('batch_size', 128),
        epochs=params_fit.get('epochs', 5),
        verbose=params_fit.get('verbose', 1),
        validation_data=params_fit.get('calidation_data', (X_train, y_train)),
        callbacks =[tensorboard_callback]
    )

    return model
    

In [0]:
def predict(model_trained, X_test, y_test, scoring = accuracy_score ):
   y_test_norm = np.argmax(y_test, axis=1)
   
   y_pred_prob = model_trained.predict(X_test) 
   y_pred = np.argmax(y_pred_prob, axis=1)

   return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding = 'same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),


  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),


  Dense(1024, activation='relu'),
  Dropout(0.3),


  Dense(num_classes, activation='softmax'),
])

In [27]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model,X_train, y_train, params_fit={}  )
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.6834 - accuracy: 0.2654 - val_loss: 1.4732 - val_accuracy: 0.5146
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 1.0805 - accuracy: 0.6486 - val_loss: 0.2972 - val_accuracy: 0.9124
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.4539 - accuracy: 0.8566 - val_loss: 0.1320 - val_accuracy: 0.9595
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2870 - accuracy: 0.9111 - val_loss: 0.0656 - val_accuracy: 0.9805
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2193 - accuracy: 0.9349 - val_loss: 0.0490 - val_accuracy: 0.9864


0.9596371882086168

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding = 'same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),


  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_one']),


  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_two']),


  Dense(num_classes, activation='softmax'),
])

In [0]:
def func_obj(params):
   model = get_model(params)
   model.compile(loss = 'categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
   
   model.fit(
        X_train, 
        y_train,
        batch_size= int(params.get('batch_size', 128)),
        epochs=5,
        verbose=0
   )
   
   score = model.evaluate(X_test, y_test, verbose = 0)
   accuracy = score[1]
   print(params, 'accuracy={}'.format(accuracy))

   return {'loss': -accuracy, 'status':STATUS_OK, 'model':model}

In [68]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one':   hp.uniform('dropout_cnn_block_one',0.3,  0.5   ),
    'dropout_cnn_block_two':   hp.uniform('dropout_cnn_block_two',0.3,  0.5 ),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',0.3,  0.5 ),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one',0.3,  0.7 ),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',0.3,  0.7 ),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_one': 0.39041592127725655, 'dropout_cnn_block_three': 0.32172882198988656, 'dropout_cnn_block_two': 0.4822658401973463, 'dropout_dense_block_one': 0.6517576640746753, 'dropout_dense_block_two': 0.3246976300705422}
accuracy=0.938095211982727
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.4904011389079694, 'dropout_cnn_block_three': 0.331578342928671, 'dropout_cnn_block_two': 0.43439063526953114, 'dropout_dense_block_one': 0.49046544563688965, 'dropout_dense_block_two': 0.352501329828361}
accuracy=0.9428571462631226
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.36442094155089, 'dropout_cnn_block_three': 0.41945339428136214, 'dropout_cnn_block_two': 0.30464402183611256, 'dropout_dense_block_one': 0.5951454295036072, 'dropout_dense_block_two': 0.38032441400983547}
accuracy=0.962358295917511
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.42457542758573097, 'dropout_cnn_block_three': 0.4731002784589675, 'dropout_cnn_block_two': 0.30535921356223